In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os
import gymnasium as gym
from gymnasium import spaces
from scipy.special import softmax
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import SubprocVecEnv, VecNormalize
from sklearn.preprocessing import MinMaxScaler
import torch as th
import sys
import tensorflow as tf
import random


# Instead of importing, copy the SimpleOHLCGenerator class 
# that you shared earlier into your current file
# Or use this simpler import:
sys.path.append("/Users/newuser/Projects/robust_algo_trader/drl/")
from ohlc_generator import SimpleOHLCGenerator  
from portfolio_env import *  

In [ ]:

trained_model

In [7]:
# EVALUATE

# trained_model = PPO.load("/Users/newuser/Projects/robust_algo_trader/drl/models/model_20250320_0441/ppo_portfolio_final")
# trained_model = PPO.load("/Users/newuser/Projects/robust_algo_trader/drl/models/model_20250319_1746/ppo_portfolio_final")
trained_model = PPO.load("/Users/newuser/Projects/robust_algo_trader/drl/models/model_20250320_1452/ppo_portfolio_final")

eval_instrument_list = ["IWM", "GS", "NEE", "PFE", "QQQ", "SPY", "VZ", "WMT", "XLE", "XLF"]    
# eval_instrument_list = ["AAPL", "MSFT", "JNJ", "PG", "JPM", "NVDA", "AMD", "TSLA", "CRM", "AMZN"] 

# eval_instrument_list = ["BRK.B", "V", "UNH", "HD", "PEP", "MA", "DIS", "CSCO", "INTC", "VZ"]
# eval_instrument_list = ["GOOG", "META", "ADBE", "NFLX", "PYPL", "SHOP",  "DIS", "CSCO", "INTC", "VZ"]
# eval_instrument_list = ["GM", "XOM", "MMM", "DOW", "F", "HAL", "CLF", "NUE", "CAH", "LYB"]
# eval_instrument_list = [  "DIA", "VTI", "IJH", "XLRE", "XLV", "XLK", "XLI", "XLU", "XLP", "AMLP"]
# eval_instrument_list = [ "MRK", "DUK", "MCD", "SO", "D", "AEP", "ED", "PEP", "KMB", "CL"]
# eval_instrument_list = [ "CRM", "ORCL", "ACN", "IBM", "SAP", "COST", "TGT", "LLY", "ABT", "TMO"]
# eval_instrument_list = ["HON", "DE", "ITW", "ETN", "GE", "BLK", "AXP", "CME", "ICE", "CB"]
# eval_instrument_list = [ "BLK", "AXP", "CME", "ICE", "CB", "JNJ", "KO", "PG", "MMM", "WMT"]
# eval_instrument_list = ["HON", "DE", "ITW", "ETN", "GE","JNJ", "KO", "PG", "MMM", "WMT"]
    
eval_stock_data_list = get_stock_data_list(eval_instrument_list)
print("Evaluating model...")
evaluate_model(eval_stock_data_list, trained_model)
print("Evaluation complete!")

Loaded IWM with 2313 data points
Loaded GS with 2313 data points
Loaded NEE with 2313 data points
Loaded PFE with 2313 data points
Loaded QQQ with 2313 data points
Loaded SPY with 2313 data points
Loaded VZ with 2317 data points
Loaded WMT with 2317 data points
Loaded XLE with 2313 data points
Loaded XLF with 2313 data points
Evaluating model...
Evaluating agent over 10 episodes with forward-looking metrics...
Testing with stocks: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Aligned all test stocks to length 2060 (multiple of 20)
Testing with stocks: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Aligned all test stocks to length 2060 (multiple of 20)
Testing with stocks: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Aligned all test stocks to length 2060 (multiple of 20)
Testing with stocks: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Aligned all test stocks to length 2060 (multiple of 20)
Standard policy evaluation - Mean reward: 275.9591 ± 2.3979
Running forward-looking evaluation...
Testing with stocks: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Alig

<Figure size 1200x600 with 0 Axes>